In [7]:
import pandas as pd
import pdb
import numpy as np
import itertools
from snakemake.io import expand
import yaml
import os

from utils import *
from sm_utils import *

In [8]:
config_file = 'config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

## 231020 pipeline restructuring -- get bws for each mouse

In [11]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC'}

config_tsv = '231020_config.tsv'
# config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
# meta_tsv = 'mouse_test_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_humanized=False)

In [15]:
# expand(get_cfg_entries(wc, df,
#                         config['merge']['bw']),
#        strand=['fwd', 'rev'])

In [18]:
expand(expand(config['merge']['bw'],
        zip,
        study=df.study.tolist(),
        genotype=df.genotype.tolist(),
        sex=df.sex.tolist(),
        age=df.age.tolist(),
        tissue=df.tissue.tolist(),
        biorep_num=df.biorep_num.tolist(),
        allow_missing=True),
        strand=['fwd', 'rev'])

['data/merge/ad002/5xFADHEMI_M_18_months_CTX_1_fwd.bw',
 'data/merge/ad002/5xFADHEMI_M_18_months_CTX_1_rev.bw',
 'data/merge/ad002/5xFADHEMI_M_18_months_CTX_1_fwd.bw',
 'data/merge/ad002/5xFADHEMI_M_18_months_CTX_1_rev.bw',
 'data/merge/ad002/5xFADWT_M_18_months_CTX_1_fwd.bw',
 'data/merge/ad002/5xFADWT_M_18_months_CTX_1_rev.bw',
 'data/merge/ad002/5xFADWT_M_18_months_CTX_1_fwd.bw',
 'data/merge/ad002/5xFADWT_M_18_months_CTX_1_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_1_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_1_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_1_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_1_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_2_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_2_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_2_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_2_rev.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_3_fwd.bw',
 'data/merge/ad003/5xFADHEMI_F_4_months_HC_3_rev.bw',
 'data/merge/ad003/5

## 231020 pipeline restructuring -- sanitizing humanized input

In [3]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'cerberus_run': '2'}

config_tsv = '231020_config.tsv'
# config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
# meta_tsv = 'mouse_test_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe,
                       include_humanized=False)

In [4]:
df[['study', 'genotype', 'sex', 'age', 'tissue', 'humanized']].drop_duplicates()

,study,genotype,sex,age,tissue,humanized
0,ad002,5xFADHEMI,M,18_months,CTX,False
2,ad002,5xFADWT,M,18_months,CTX,False
4,ad003,5xFADHEMI,F,4_months,HC,False
10,ad003,5xFADWT,F,4_months,HC,False
16,ad005,5xBIN1_HO,F,4_months,HC,False
22,ad005,BIN1_HO,F,4_months,HC,False
29,ad006,C57B6J,F,8_weeks,HC,False
35,ad006,C57B6J,M,8_weeks,HC,False
41,ad007,ABCA7_HO,F,4_months,HC,False


## 231020 pipeline restructuring -- more cerberus shenannigans

In [81]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'cerberus_run': 2}
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC'}

# config_tsv = '231018_config.tsv'
config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

In [83]:
get_cfg_entries({'cerberus_run':df.cerberus_run.unique().tolist()},
                        df,
                        config['cerberus']['ca'])

['data/cerberus/ad003/5xFADHEMI_F_4_months_HC_1.h5',
 'data/cerberus/ad003/5xFADWT_F_4_months_HC_2.h5',
 'data/cerberus/ad005/BIN1_HO_F_4_months_HC_3.h5']

In [74]:
# need to get tss / tes by adding end mode wcs
# expand(['cerberus']['agg']['ends'],
#                                end_mode='tss')[0]

In [76]:
expand(get_cfg_entries(wc,
                        df,
                        config['cerberus']['agg']['ends']),
       end_mode='tss')[0]

['data/cerberus/agg/ad003_5xFADHEMI_F_4_months_HC_1_tss.bed']

## 231019 pipeline restructuring -- cerberus getting previous cerberus run

In [50]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'cerberus_run': '2'}

# config_tsv = '231018_config.tsv'
config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

In [51]:
df.cerberus_run.max()

'3'

In [60]:
get_cfg_entries({'cerberus_run':df.cerberus_run.max()},
                        df,
                        config['cerberus']['agg']['ends'])

['data/cerberus/agg/ad005_BIN1_HO_F_4_months_HC_3_{end_mode}.bed']

In [57]:
def get_prev_cerb_entry(wc, df, cfg_entry):
    """
    Get the previous config entry run for Cerberus. Ensure that 
    only one file meets these criteria.
    """
    prev_wc = {'cerberus_run': str(int(wc['cerberus_run'])-1)}
    file = get_cfg_entries(prev_wc, df, cfg_entry)
    assert len(file) == 1
    file = file[0]
    
    # add in the end mode if we have it
    if 'end_mode' in wc.keys():
        file = expand(file, 
                      zip,
                      end_mode=wc['end_mode'])
        assert len(file) == 1
        file = file[0]
    
    return file

In [58]:
get_prev_cerb_entry(wc, df, config['cerberus']['agg']['ics'])

'data/cerberus/agg/ad003_5xFADHEMI_F_4_months_HC_1_ic.tsv'

In [59]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'end_mode': 'tss',
      'cerberus_run': '2'}

# config_tsv = '231018_config.tsv'
config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

get_prev_cerb_entry(wc, df, config['cerberus']['agg']['ends'])

'data/cerberus/agg/ad003_5xFADHEMI_F_4_months_HC_1_tss.bed'

In [8]:
prev_wc = {'cerberus_run': str(int(wc['cerberus_run'])-1)}
prev_wc

get_cfg_entries({'cerberus_run': str(int(wc['cerberus_run'])-1)}, df, config['cerberus']['ics'])

['data/cerberus/ad003/5xCLU-h2kbKI_HO_F_4_months_HC_1_ic.tsv']

In [28]:
get_cfg_entries({'cerberus_run':df.cerberus_run.max()},
                        df,
                        config['cerberus']['agg']['ics'])

['data/cerberus/agg/ad005_BIN1_HO_F_4_months_HC_3_ic.tsv']

## 231019 pipeline restructuring -- cerberus getting source from current wcs (should be the study_genotype_sex_age_tissue combo again)

In [6]:
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': '4_months',
      'tissue': 'HC',
      'end_mode': 'tss',
      'cerberus_run': '2'}

config_tsv = '231018_config.tsv'
# config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)

In [7]:
get_df_col(wc, df, 'source')

'ad003_5xFADHEMI_F_4_months_HC'

## 231019 pipeline restructuring -- cerberus stuff

In [118]:
config_tsv = '231018_config.tsv'
# config_tsv = 'test_local_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)


In [130]:
# assign a cerberus run to each "sample" (study+genotype+sex+age+tissue)
# but first sort on study and sample such that they will always be ordered in the same way
# this should freeze our results
gb_cols = ['study', 'genotype', 'sex', 'age', 'tissue']
df = df.sort_values(by=gb_cols, ascending=True)
temp = df.copy(deep=True)
temp = temp[gb_cols].groupby(gb_cols).count().reset_index()
temp['cerberus_run'] = [i+1 for i in range(len(temp.index))]
df = df.merge(temp, how='left', on=gb_cols)

In [132]:
df.head(10)

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset,cerberus_run
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12517_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12517_lig-blk_1,1.0,ad003_12517,12517,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,1,5xCLU-h2kbKI_HO_F_4_months_HC_1,1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12517_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_12517_lig-blk_2,1.0,ad003_12517,12517,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,1,5xCLU-h2kbKI_HO_F_4_months_HC_1,1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12649_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12649_lig-blk_1,1.0,ad003_12649,12649,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,2,5xCLU-h2kbKI_HO_F_4_months_HC_2,1
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12649_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_12649_lig-blk_2,1.0,ad003_12649,12649,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,2,5xCLU-h2kbKI_HO_F_4_months_HC_2,1
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12659_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_12659_lig-blk_1,1.0,ad003_12659,12659,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,3,5xCLU-h2kbKI_HO_F_4_months_HC_3,1
5,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_12659_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_12659_lig-blk_2,1.0,ad003_12659,12659,ad003,5xCLU-h2kbKI_HO,F,4_months,HC,5xCLU-h2kbKI_HO_F_4_months_HC,3,5xCLU-h2kbKI_HO_F_4_months_HC_3,1
6,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,2
7,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,2
8,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2,2
9,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2,2


## 231019 pipeline restructuring -- lapa filtering settings

In [105]:
t_novs = config['lapa']['filt']['t_novs']
g_novs = config['lapa']['filt']['g_novs']
filt_spikes = config['lapa']['filt']['remove_spikes']

In [109]:
type(filt_spikes)

bool

## 231019 pipeline restructuring -- lapa input

In [91]:
def subset_df_on_wcs(wc, df):
    """
    Return a copy of the input metadata df limited to the wildcards
    """
    temp = df.copy(deep=True)
    for key, item in wc.items():
        
        # for entries that we don't have in the df, such as LAPA end mode
        if key not in df.columns:
            continue
        
        # if we're given a list of possibilities
        if type(item) == list:
            temp = temp.loc[temp[key].isin(item)]
        else:
            temp = temp.loc[temp[key] == item]
    return temp

In [92]:
def get_cfg_entries(wc, df, cfg_entry, return_df=False):
    """
    Expand a config entry based on the wildcards and the
    values in df that satisfy these wildcards

    Parameters:
        return_df (bool): Return DataFrame with 'file' column
            as opposed to list of files. Default: False
    """
    temp = subset_df_on_wcs(wc, df)

    # cols = ['study', 'genotype', 'sex',
    #         'age', 'tissue', 'biorep_num',
    #         'flowcell']

    study = temp.study.tolist()
    genotype = temp.genotype.tolist()
    sex = temp.sex.tolist()
    age = temp.age.tolist()
    tissue = temp.tissue.tolist()
    biorep_num = temp.biorep_num.tolist()
    flowcell = temp.flowcell.tolist()

    files = expand(cfg_entry,
                   zip,
                   study=study,
                   genotype=genotype,
                   sex=sex,
                   age=age,
                   tissue=tissue,
                   biorep_num=biorep_num,
                   flowcell=flowcell,
                   allow_missing=True)

    temp['file'] = files

    # make sure we only take unique ones
    # cols.append('file')
    # temp = temp[cols]
    temp = temp.drop_duplicates(subset='file', keep='first')
    files = temp['file'].tolist()

    if return_df:
        return temp
    else:
        return files

In [101]:
def get_lapa_settings(wc, cfg_entry, kind):
    """
    Get the command name or file output name
    given whether we're running LAPA in tss or tes mode

    Parameters:
        lapa_ends (str): Config entry 
        kind (str): {'temp_file', 'lapa_cmd'}
    """
    
    # get the output file from the lapa run
    lapa_ends = get_cfg_entries(wc, df, cfg_entry, return_df=False)
    
    # now expand for end mode
    lapa_ends = expand(lapa_ends, 
                       zip=True,
                       end_mode=wc['end_mode'])    
    assert len(list(set(lapa_ends))) == 1
    lapa_ends = lapa_ends[0]
    
    if kind == 'temp_file':
        temp = os.path.dirname(lapa_ends)+'/'
        if wc['end_mode'] == 'tes':
            temp += 'polyA_clusters.bed'
        elif wc['end_mode'] == 'tss':
            temp += 'tss_clusters.bed'
        return temp
    elif kind == 'lapa_cmd':
        if wc['end_mode'] == 'tes':
            return 'lapa'
        elif wc['end_mode'] == 'tss':
            return 'lapa_tss'

In [102]:
# get all files from bioreps of same genotype, sex, age, tissue
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': ['4_months', '10_days'],
      'tissue': 'HC',
      'end_mode': 'tss'}

cfg_entry = config['lapa']['ends']
get_lapa_settings(wc, cfg_entry, 'temp_file')

'data/lapa/ad003/tss/tss_clusters.bed'

In [103]:
get_lapa_settings(wc, cfg_entry, 'lapa_cmd')

'lapa_tss'

## 231019 pipeline restructuring -- talon input

In [71]:
# utilities related to defining snakemake rules
from snakemake.io import expand

def subset_df_on_wcs(wc, df):
    """
    Return a copy of the input metadata df limited to the wildcards
    """
    temp = df.copy(deep=True)
    for key, item in wc.items():
        if type(item) == list:
            temp = temp.loc[temp[key].isin(item)]
        else:
            temp = temp.loc[temp[key] == item]
    return temp

def get_df_col(wc, df, col):
    """
    From the metadata dataframe df, get the entries that satisfy
    the wildcards requirements and return the corresponding value
    from col. Ensure that this is always a 1:1 relationship, otherwise
    throw an error.
    """
    cols = [col] + [key for key, item in wc.items()]

    temp = subset_df_on_wcs(wc, df)
    temp = temp[cols].drop_duplicates()

    if len(temp.index) != 1:
        msg = 'Issues getting data from DF with wildcards'
        for key, item in wc.items():
            msg+=f'\n{key}: {item}'
        raise ValueError(msg)

    val = temp[col].tolist()[0]
    return val

def get_cfg_entries(wc, df, cfg_entry, return_df=False):
    """
    Expand a config entry based on the wildcards and the
    values in df that satisfy these wildcards
    
    Parameters:
        return_df (bool): Return DataFrame with 'file' column 
            as opposed to list of files. Default: False
    """
    temp = subset_df_on_wcs(wc, df)

    # cols = ['study', 'genotype', 'sex',
    #         'age', 'tissue', 'biorep_num',
    #         'flowcell']

    study = temp.study.tolist()
    genotype = temp.genotype.tolist()
    sex = temp.sex.tolist()
    age = temp.age.tolist()
    tissue = temp.tissue.tolist()
    biorep_num = temp.biorep_num.tolist()
    flowcell = temp.flowcell.tolist()

    files = expand(cfg_entry,
                   zip,
                   study=study,
                   genotype=genotype,
                   sex=sex,
                   age=age,
                   tissue=tissue,
                   biorep_num=biorep_num,
                   flowcell=flowcell)

    temp['file'] = files

    # make sure we only take unique ones
    # cols.append('file')
    # temp = temp[cols]
    temp = temp.drop_duplicates(subset='file', keep='first')
    files = temp['file'].tolist()
    
    if return_df:
        return temp
    else: 
        return files

In [110]:
# get all files from bioreps of same genotype, sex, age, tissue
wc = {'study': 'ad003',
      'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': ['4_months', '10_days'],
      'tissue': 'HC'}

files = get_cfg_entries(wc, df, config['merge']['sort_bam'])
files
temp = get_cfg_entries(wc, df, cfg_entry, return_df=True)
temp

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset,file
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,data/lapa/ad003/{end_mode}/5xFADHEMI_F_4_month...


In [112]:
temp = subset_df_on_wcs(wc, df)
temp


,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3
5,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11625_lig-blk_2,1.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3


In [79]:
temp = df.loc[(df.study==wc['study'])&\
       (df.genotype==wc['genotype'])&\
       (df.sex==wc['sex'])&\
       (df.age.isin(wc['age']))&\
       (df.tissue==wc['tissue'])]
temp

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3
5,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11625_lig-blk_2,1.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3


In [74]:
temp[['sample', 'dataset', 'platform', 'file']]

,sample,dataset,platform,file
0,5xFADHEMI_F_4_months_HC,5xFADHEMI_F_4_months_HC_1,ONT,data/flowcell_merge/5xFADHEMI_F_4_months_HC_1.bam
2,5xFADHEMI_F_4_months_HC,5xFADHEMI_F_4_months_HC_2,ONT,data/flowcell_merge/5xFADHEMI_F_4_months_HC_2.bam
4,5xFADHEMI_F_4_months_HC,5xFADHEMI_F_4_months_HC_3,ONT,data/flowcell_merge/5xFADHEMI_F_4_months_HC_3.bam


In [75]:
temp

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset,file
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,data/flowcell_merge/5xFADHEMI_F_4_months_HC_1.bam
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2,data/flowcell_merge/5xFADHEMI_F_4_months_HC_2.bam
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3,data/flowcell_merge/5xFADHEMI_F_4_months_HC_3.bam


In [117]:
df.loc[df.study=='ad005']

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset
24,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13051_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13051_lig-blk_2,1.0,ad005_13051,13051,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,3,BIN1_HO_F_4_months_HC_3
25,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13036_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13036_lig-blk_2,1.0,ad005_13036,13036,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,1,BIN1_HO_F_4_months_HC_1
26,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13047_lig-blk_3_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13047_lig-blk_3,1.0,ad005_13047,13047,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,2,BIN1_HO_F_4_months_HC_2
27,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13051_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad005_13051_lig-blk_1,1.0,ad005_13051,13051,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,3,BIN1_HO_F_4_months_HC_3
28,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13053_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13053_lig-blk_1,1.0,ad005_13053,13053,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,3,5xBIN1_HO_F_4_months_HC_3
29,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13022_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13022_lig-blk_2,1.0,ad005_13022,13022,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,2,5xBIN1_HO_F_4_months_HC_2
30,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13053_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad005_13053_lig-blk_2,1.0,ad005_13053,13053,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,3,5xBIN1_HO_F_4_months_HC_3
31,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13019_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad005_13019_lig-blk_1,1.0,ad005_13019,13019,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,1,5xBIN1_HO_F_4_months_HC_1
32,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13047_lig-blk_2_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad005_13047_lig-blk_2,1.0,ad005_13047,13047,ad005,BIN1_HO,F,4_months,HC,BIN1_HO_F_4_months_HC,2,BIN1_HO_F_4_months_HC_2
33,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad005_13022_lig-blk_1_t1.fastq.gz,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad005_13022_lig-blk_1,1.0,ad005_13022,13022,ad005,5xBIN1_HO,F,4_months,HC,5xBIN1_HO_F_4_months_HC,2,5xBIN1_HO_F_4_months_HC_2


## 231017 pipeline restructuring

In [4]:
configfile: 'config.yml'
config_tsv = '231018_config.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True
df = parse_config_file(config_tsv,
                       meta_tsv,
                       auto_dedupe=auto_dedupe)


In [13]:
wc = {'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': ['4_months', '10_days'],
      'tissue': 'HC',
      'biorep_num': '1',
      'flowcell': '1'}

In [41]:
cfg_entry = config['talon_label']['sort_bam']
wc = {'genotype': '5xFADHEMI',
      'sex': 'F',
      'age': ['4_months', '10_days'],
      'tissue': 'HC',
      'biorep_num': '1'}

temp = subset_df_on_wcs(wc, df)
study = temp.study.tolist()
genotype = temp.genotype.tolist()
sex = temp.sex.tolist()
age = temp.age.tolist()
tissue = temp.tissue.tolist()
biorep_num = temp.biorep_num.tolist()
flowcell = temp.flowcell.tolist()

files = expand(cfg_entry,
               zip,
               study=study,
               genotype=genotype,
               sex=sex,
               age=age,
               tissue=tissue,
               biorep_num=biorep_num,
               flowcell=flowcell)
return files

['data/talon_label/5xFADHEMI_F_4_months_HC_1_1_labeled_sorted.bam',
 'data/talon_label/5xFADHEMI_F_4_months_HC_1_2_labeled_sorted.bam']

In [40]:
df.head()

,batch,platform,fname,basename,path,flowcell,file_stem,chop_num,sample_temp,mouse_id,study,genotype,sex,age,tissue,sample,biorep_num,dataset
0,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
1,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1
2,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
3,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2
4,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,ad003,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3


In [30]:
get_df_col(wc, df, 'fname')

'/share/crsp/lab/seyedam/share/ad_nanopore_tmp/AD003/ad003_trimfastq/ad003_11616_lig-blk_1_t1.fastq'

In [25]:
# replicates in modelad consist of 
# * genotype
# * sex
# * age
# * tissue